In [1]:
import config as cfg
import utils.helper_functions as hf
import utils.inputs as inputs
import utils.outputs as outputs
import utils.tf_model as tf_model
import utils.evaluate as eval

from IPython.display import display, clear_output

import pandas as pd
import numpy as np

import hyperopt
from hyperopt import hp, fmin, tpe

import os

os.environ['PYDEVD_DISABLE_FILE_VALIDATION'] = '1' # disable file validation in the debugger
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1' #0: All logs (default setting), 1: Filter out INFO logs, up to 3
pd.options.mode.copy_on_write = True # avoid making unnecessary copies of DataFrames or Series


# import config as cfg
# import utils.helper_functions as hf

# import tensorflow as tf
# from tensorflow.keras.models import Sequential # type: ignore
# from tensorflow.keras.layers import Dense, BatchNormalization, Dropout # type: ignore

# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import StandardScaler

# from collections import Counter

# import os
# os.environ['PYDEVD_DISABLE_FILE_VALIDATION'] = '1' # disable file validation in the debugger
# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1' #0: All logs (default setting), 1: Filter out INFO logs, up to 3


In [ ]:
if cfg.use_hyperopt:
    print(cfg.hyperopt_n_iterations)
else:
    num_combinations = hf.get_num_combinations(cfg.param_grid) 

In [ ]:
df = pd.read_pickle(cfg.db_path)
df = hf.get_rows_after_date(df, cfg.start_date)
df = hf.fillnavalues(df)

def get_single_level_df(df, ohlcv):
    new_df = df[[ohlcv]]
    new_df = hf.remove_top_column_name(new_df)

    return new_df

def get_ohlcv_dfs(df):
    df_open = get_single_level_df(df, 'Open')
    df_high = get_single_level_df(df, 'High')
    df_low = get_single_level_df(df, 'Low')
    df_close = get_single_level_df(df, 'Close')
    df_volume = get_single_level_df(df, 'Volume')
    
    return {'df_open': df_open, 'df_high': df_high, 'df_low': df_low,
            'df_close': df_close, 'df_volume': df_volume}

num_tickers = hf.get_num_tickers(get_single_level_df(df, 'Open'))
print(f'number of tickers: {num_tickers}')


In [4]:
def get_df_data(hyperparams):
    df_buy = get_single_level_df(df, hyperparams['buying_time'])
    df_sell = get_single_level_df(df, hyperparams['selling_time'])
    dfs_ohlcv = get_ohlcv_dfs(df)

    if os.path.exists(cfg.transformed_data_path) and cfg.use_saved_transformed_data:
        df_data = pd.read_pickle(cfg.transformed_data_path)
        print(f'using existing {cfg.transformed_data_path}')
    else:
        print(f'need to create {cfg.transformed_data_path}')
        df_data = inputs.get_inputs(df_buy, dfs_ohlcv, hyperparams['buying_time'])
        
        df_data.to_pickle(cfg.transformed_data_path)
        print(f'saved new {cfg.transformed_data_path}')

    df_data = outputs.add_outputs(df_data, df_buy, df_sell, dfs_ohlcv, num_tickers, cfg.output_class_name, cfg.fee, **hyperparams)

    df_data = df_data.dropna()

    return df_data

In [5]:


# def get_dfs_input_output(df_data, output_class_name):
#     input_columns = [col for col in df_data.columns if col.startswith('input_')]
#     df_input = df_data[input_columns]
#     df_output = df_data[[output_class_name]]

#     return df_input, df_output

# def get_test_train_data(df_input, df_output, test_size):
#     X_train = df_input[:-test_size].values
#     y_train = df_output[:-test_size].values.ravel().astype(int)

#     X_test = df_input.tail(test_size).values
#     y_test = df_output.tail(test_size).values.ravel().astype(int)

#     scaler = StandardScaler()
#     X_train = scaler.fit_transform(X_train)
#     X_test = scaler.transform(X_test)

#     hf.save_object(scaler, './outputs/scaler.pkl')

#     print(f"number of elements in y_train: {len(y_train)}")
#     print(f"number of elements in y_test: {len(y_test)}")

#     return {'X_train': X_train, 'X_test': X_test, 'y_train': y_train, 'y_test': y_test}

# def create_model(**kwargs):
#     X_train = kwargs.get('X_train')
#     X_test = kwargs.get('X_test')
#     y_train = kwargs.get('y_train')
#     y_test = kwargs.get('y_test')

#     thresholds = kwargs.get('thresholds')
    
#     size_layer_1 = kwargs.get('size_layer_1')
#     size_layer_2 = kwargs.get('size_layer_2')
#     size_layer_3 = kwargs.get('size_layer_3')
#     dropout_rate = kwargs.get('dropout_rate')
#     balance_data = kwargs.get('balance_data')
#     batch_size = kwargs.get('batch_size')

#     last_layers_size = len(thresholds) + 1

#     model = Sequential()

#     model.add(Dense(size_layer_1, input_shape=(X_train.shape[1],), activation='relu'))
#     model.add(BatchNormalization())
#     model.add(Dropout(dropout_rate))
#     model.add(Dense(size_layer_2, activation='relu'))
#     model.add(BatchNormalization())
#     model.add(Dropout(dropout_rate))
#     model.add(Dense(size_layer_3, activation='relu'))
#     model.add(BatchNormalization())
#     model.add(Dropout(dropout_rate))
#     model.add(Dense(last_layers_size, activation='softmax'))

#     model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

#     if (balance_data):
#         counter = Counter(y_train)
#         max_count = max(counter.values())
#         class_weights = {cls: max_count / count for cls, count in counter.items()}
#         model.fit(X_train, y_train, epochs=cfg.epochs, batch_size=batch_size, validation_data=(X_test, y_test), class_weight=class_weights)
#     else:
#         model.fit(X_train, y_train, epochs=cfg.epochs, batch_size=batch_size, validation_data=(X_test, y_test))

#     model.save(cfg.model_path)

# def load_model(df_data, hyperparams):
#     df_input, df_output = get_dfs_input_output(df_data, cfg.output_class_name)

#     test_train_data = get_test_train_data(df_input, df_output, cfg.test_size)

#     if os.path.exists(cfg.model_path) and cfg.use_saved_model:
#         print(f'using existing {cfg.model_path}')
#     else:
#         print(f'need to create {cfg.model_path}')
#         create_model(**{**test_train_data, **hyperparams})
    
#     model = tf.keras.models.load_model(cfg.model_path)

#     return test_train_data, model

In [ ]:
from itertools import product

i = 0
results = []

def objective(hyperparams):
    hyperparams['thresholds'] = [hyperparams['thresholds']]
    hyperparams['rank_pct_thresholds'] = [hyperparams['rank_pct_thresholds']]
    
    df_data = get_df_data(hyperparams)
    test_train_data, model = model.load_model(df_data, hyperparams)
    performance_metrics = eval.evaluate_model(df_data, model, test_train_data, hyperparams)

    result = {**performance_metrics, **hyperparams, 'epochs': cfg.epochs}
    print(result)
    results.append(result)

    performance = result['performance_score']

    return -performance

if cfg.use_hyperopt:
    best = fmin(objective, cfg.search_space, algo=tpe.suggest, max_evals=cfg.hyperopt_n_iterations)
    print(f'best parameters: {best}')
else:
    for params in product(*cfg.param_grid.values()):
        i += 1
        # clear_output(wait=True) # clear printed outputs
        hf.print_combination(i, num_combinations)

        hyperparams = dict(zip(cfg.param_grid.keys(), params))

        df_data = get_df_data(hyperparams)
        test_train_data, model = tf_model.load_model(df_data, hyperparams)
        performance_metrics = eval.evaluate_model(df_data, model, test_train_data, num_tickers, num_combinations, hyperparams)

        result = {**performance_metrics, **hyperparams, 'epochs': cfg.epochs}
        print(result)
        results.append(result)


In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

df_results = pd.DataFrame(results)
df_results = df_results.sort_values(by='performance_score', ascending=False)
df_results.head(1000)

In [7]:
df_results.to_excel(f'./outputs/{hf.get_date()}_classifier_results.xlsx')